In [ ]:
import sys
from os.path import join
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
#from tensorflow.python.keras.applications import ResNet50

from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils, to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import ResNet50

import os


In [ ]:
# Convoluted Base MODEL

conv_base = ResNet50(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3))
print(conv_base.summary())

In [ ]:
# MODEL

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))

print(model.summary())

In [ ]:

# Make the conv_base NOT trainable:

for layer in conv_base.layers[:]:
   layer.trainable = False

print('conv_base is now NOT trainable')


In [ ]:
# Compile frozen conv_base + my top layer

model.compile(loss='categorical_crossentropy',
                  optimizer= optimizers.Adam(),
                  metrics=['accuracy'])
print("model compiled")
print(model.summary())

Leer datos

In [ ]:
# dataset1 = pd.read_csv("/content/drive/MyDrive/Dataton/finaldf.csv")
dataset1 = pd.read_csv("/content/drive/MyDrive/Dataton/labels_dataset1_v2.csv")
dataset1.drop_duplicates(inplace=True)
dataset1.reset_index(inplace=True,drop=True)
dataset1['image_name'] = [str(i) + '.jpg' for i in range(len(dataset1))]
print(dataset1.label.value_counts())
dataset1

In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)
    return dict(zip(class_labels, class_weights))
 

In [ ]:
class_weights = generate_class_weights(dataset1['label'])
class_weights


Crear conjunto test,train,val

In [ ]:
# import random
# a = random.randint(0,500)
# print(a)
# random_state = a

from sklearn.model_selection import train_test_split
random_state = 261
X_train, X_test = train_test_split(dataset1,test_size=0.10, random_state=random_state)
X_train, X_val = train_test_split(X_train, test_size=0.25, random_state=random_state)

directory = '/content/drive/MyDrive/Dataton/datos/all'
batch_size = 16
target_size = (224,224)
datagen=ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(
    dataframe=X_train,
    directory=directory,
    x_col="image_name",
    y_col="label",
    batch_size=batch_size,
    shuffle=False,
    class_mode="categorical",
    target_size=target_size)

validation_generator=datagen.flow_from_dataframe(
    dataframe=X_val,
    directory=directory,
    x_col="image_name",
    y_col="label",
    batch_size=batch_size,
    shuffle=False,
    class_mode="categorical",
    target_size=target_size)

test_generator=datagen.flow_from_dataframe(
    dataframe=X_test,
    directory=directory,
    x_col="image_name",
    y_col="label",
    batch_size=batch_size,
    shuffle=False,
    class_mode="categorical",
    target_size=target_size)

print(validation_generator.image_shape)

In [ ]:
# Short training ONLY my top layers 
#... so the conv_base weights will not be destroyed by the random intialization of the new weights

history = model.fit(train_generator,
                    class_weight = dictio,
                    epochs=1,
                    steps_per_epoch = train_generator.samples // batch_size,
                    validation_data = validation_generator,
                    validation_steps = validation_generator.samples // batch_size)
         

In [ ]:
#probar tiempo-------------------------------------------------------
with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True)):
  history = model.fit(train_generator,
                                epochs=1,
                                steps_per_epoch = train_generator.samples // batch_size,
                                validation_data = validation_generator,
                                validation_steps = validation_generator.samples // batch_size)
                                # workers = 4, use_multiprocessing = True)

In [ ]:

# Make last block of the conv_base trainable:
for layer in conv_base.layers[:165]:
   layer.trainable = False
for layer in conv_base.layers[165:]:
   layer.trainable = True

print('Last block of the conv_base is now trainable')

In [ ]:
# Compile frozen conv_base + UNfrozen top block + my top layer ... SLOW LR
# Compiling using adam and categorical crossentropy
model.compile(loss='categorical_crossentropy',
                  optimizer= optimizers.Adam(learning_rate=1e-5),
                  metrics=['accuracy'])

print(model.summary())

In [ ]:
# Long training with fine tuning

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


history = model.fit(train_generator, callbacks=[model_checkpoint_callback],
                              epochs=50,
                              steps_per_epoch = train_generator.samples // batch_size,
                              validation_data = validation_generator,
                              validation_steps = validation_generator.samples // batch_size,workers = 4, use_multiprocessing = True)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps= test_generator.samples // batch_size, verbose=1)
print('test acc:', test_acc)

In [ ]:
# SAVE or LOAD model (Keras - all batteries included: architecture, weights, optimizer, last status in training, etc.)
# YOU supply this model.h5 file from previous training session(s) - expected as a data source by Kaggle

# SAVE model
model.save('ResNet50FineTune.h5')
print("ResNet50FineTune.h5 was saved")

# LOAD model
#del model
#model = load_model('')
#print("ResNet50FineTune.h5 was loaded")